In [1]:
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from multiprocessing import Pool
import os
import logging
from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
path_data_folder = Path.cwd().parent.parent / 'data'
print(path_data_folder)

folder_raw_data_milling = path_data_folder / "raw/milling"
folder_interim_data_milling = path_data_folder / "interim/milling"
folder_processed_data_milling = path_data_folder / "processed/milling"

/home/tim/Documents/feat-store/data


In [3]:
df_raw = pd.read_csv(folder_raw_data_milling / "milling.csv.gz", compression='gzip',)
df_raw.head()

,cut_id,cut_no,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class
0,0_0,0,1,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0
1,0_0,0,1,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0
2,0_0,0,1,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0
3,0_0,0,1,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0
4,0_0,0,1,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0


In [5]:
# add column for the y label, where 1 is the label for the positive class when the tool_class > 1
df_raw['y'] = df_raw['tool_class'] > 1
df_raw['y'] = df_raw['y'].astype(int)
df_raw.head()

,cut_id,cut_no,case,time,ae_spindle,ae_table,vib_spindle,vib_table,smcdc,smcac,tool_class,y
0,0_0,0,1,0.000,0.219727,0.272827,0.733643,2.116699,6.840820,0.124512,0,0
1,0_0,0,1,0.004,0.246582,0.322266,0.778809,2.277832,6.660156,-0.561523,0,0
2,0_0,0,1,0.008,0.294189,0.283813,0.758057,2.343750,6.508789,-2.099609,0,0
3,0_0,0,1,0.012,0.323486,0.260010,0.726318,2.448730,6.542969,-2.731934,0,0
4,0_0,0,1,0.016,0.290527,0.253296,0.653076,2.546387,6.621094,-3.505859,0,0


In [29]:
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [30]:
df = df[df['case']==15]

df['y'] = df['tool_class'] > 1
df['y'] = df['y'].astype(int)
print(df.shape)
df.head()

(520, 4511)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
10654,154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,...,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592,154,15,0,0
10655,154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,...,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873,154,15,0,0
10656,154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,...,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656,154,15,0,0
10657,154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,...,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192,154,15,0,0
10658,154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,...,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448,154,15,0,0


In [31]:
# drop any columns with NaN values
df = df.dropna(axis=1, how='any')
df.set_index('cut_id', inplace=True)

print(df.shape)
df.head()

(520, 2803)


,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
cut_id,,,,,,,,,,,,,,,,,,,,,
154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,1.0,...,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592,154,15,0,0
154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,0.0,...,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873,154,15,0,0
154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,1.0,...,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656,154,15,0,0
154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,1.0,...,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192,154,15,0,0
154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,1.0,...,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448,154,15,0,0


In [35]:
df = df.drop(['case', 'tool_class', 'y', 'cut_no'], axis=1)
df.head()

,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,smcac__permutation_entropy__dimension_4__tau_1,smcac__permutation_entropy__dimension_5__tau_1,smcac__permutation_entropy__dimension_6__tau_1,smcac__permutation_entropy__dimension_7__tau_1,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98"
cut_id,,,,,,,,,,,,,,,,,,,,,
154_0,3.277600e-05,3.138502e-07,-0.000004,0.002755,0.002738,0.002713,9.108564,0.103335,0.0,1.0,...,1.947155,2.458517,2.871755,3.184946,1.210316,9.495060,3.155575,1.471686,1.409514,4.198592
154_1,-3.481216e-05,-4.725636e-05,-0.000036,0.002773,0.002761,0.002748,9.774098,0.106730,0.0,0.0,...,2.074538,2.553512,2.975967,3.298287,1.145610,6.663433,1.933154,1.306129,1.204335,1.378873
154_10,-4.339720e-07,2.743999e-06,0.000019,0.002787,0.002761,0.002742,9.451535,0.103611,0.0,1.0,...,1.908081,2.360681,2.757294,3.136191,1.267046,11.902700,5.012953,4.047941,1.298777,8.912656
154_11,1.703928e-05,3.375236e-05,0.000009,0.002836,0.002819,0.002831,10.165892,0.119084,0.0,1.0,...,2.064300,2.490675,2.817898,3.131181,1.365485,12.961481,6.541192,6.282037,1.916424,10.629192
154_12,-8.043911e-06,-3.024958e-07,-0.000004,0.002694,0.002670,0.002650,8.906037,0.113346,0.0,1.0,...,2.091574,2.565142,2.939898,3.276163,1.257279,10.932990,4.016544,1.559468,1.285410,6.817448


In [32]:
dfy = df['y']
dfy.head()

cut_id
154_0     0
154_1     0
154_10    0
154_11    0
154_12    0
Name: y, dtype: int64

# Test Selection

In [6]:
from src.features.select_feat_from_dataframe import milling_select_features, milling_add_y_label_anomaly

In [70]:
# load first 500 rows of data from milling_features.csv.gz
# df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip', nrows=500)
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [80]:
df_train, df_val, df_test = milling_create_data_splits(df)

df_feat_train shape: (6942, 4510)
df_feat_test shape: (2314, 4510)
df_feat_val shape: (2314, 4510)


In [77]:
# count the number values for each 'tool_class' in df_train
df_train['tool_class'].value_counts()

1    3900
0    2536
2     506
Name: tool_class, dtype: int64

In [79]:
df_val['tool_class'].value_counts()

1    1300
0     845
2     169
Name: tool_class, dtype: int64

In [81]:
df_val[df_val['tool_class'] == 2]['case'].unique()

array([13, 11, 15, 10, 14,  5,  9])

In [69]:


df = milling_add_y_label_anomaly(df)
df = milling_select_features(df)


In [66]:
df.head()

,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,ae_spindle__symmetry_looking__r_0.05,...,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class,y
cut_id,,,,,,,,,,,,,,,,,,,,,
50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,1.0,...,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1,0
50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,1.0,...,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1,0
50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,1.0,...,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1,0
50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,1.0,...,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1,0
50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,1.0,...,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1,0


# Split on Case Number

In [4]:
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)
df.head()

(11570, 4510)


,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [5]:
for i in df['case'].unique():
    print(i)


4
9
10
11
12
3
13
14
1
2
5
6
7
8
15
16


In [6]:
int(df[df['cut_no']==7]['tool_class'].unique())

1

In [7]:
# dictionary comprehension to create dict of each cut_no and its corresponding tool_class
cut_id_tool_class_dict = {i : int(df[df['cut_no']==i]['tool_class'].unique()) for i in df['cut_no'].unique()}

In [24]:
cut_numbers = []
tool_classes = []
for i in cut_id_tool_class_dict:
    cut_numbers.append(i)
    tool_classes.append(cut_id_tool_class_dict[i])


cut_numbers_train, cut_numbers_test, tool_classes_train, tool_classes_test = train_test_split(
    np.array(cut_numbers), np.array(tool_classes),  test_size=0.6, random_state=13, stratify=np.array(tool_classes)
)

In [25]:
print(len(cut_numbers_train))
cut_numbers_train

66


array([ 49, 107,  93, 154,  36,  60,  34,  52,  44,  55,  70,  82,  67,
        84, 120, 100, 144, 147,  15,   1,   3,  69,  64, 140,   2, 139,
        85,   7, 150,  63,  47,  61,  18, 119,  62,  81,  12,  78,  14,
        43, 157,  19,  13,   9, 110,  24, 145, 101,  87,  66,  91, 111,
       105,  77, 161,  35,  51,  21, 165, 138, 118,   4, 123,  26,  80,
        37])

In [28]:
np.intersect1d(cut_numbers_train, cut_numbers_test)

array([], dtype=int64)

In [67]:
from src.features.select_feat_from_dataframe import milling_stratify_cut_no

(cut_no_train, cut_no_val, cut_no_test) = milling_stratify_cut_no(df, random_seed=76, train_size_pct=0.40)


cut_numbers_train shape: 66
cut_numbers_test shape: 50
cut_numbers_val shape: 49


In [68]:
cut_no_train

array([162,  32,  46,  29, 105,  14,  91,  18,   9, 156,  15,  84,  35,
        85,  82,  87, 125,   6,  13,  73,  50,  42, 112,  86,  41, 100,
       153,  55,  27, 131, 140,  26,  59, 126,  88,  20, 154, 145,  30,
        31, 161, 147,   0,  48,  89,  72,  60,  33,  83, 110, 122,  34,
        92, 159,  45,  77, 143,  36,  19, 141,  47, 151, 116, 157, 149,
        51])

In [64]:
cut_no_train

array([  2,  69,  45,  72,  81, 165,  37, 141, 155, 131, 121,  40,  83,
       104,  23,  66,  97, 166, 154,  79, 112, 109,  44, 119, 117, 143,
        38, 161, 115,  43, 157, 106,  18,   0,  51,  63, 160,  91,  93,
        31,  82,  87, 162,  46,  55, 123,  21,  99,  27, 148,  12,  68,
        28, 108,  33,   9,  60,  77, 156,  92,  65,  59,  50, 122,  29,
        90])

In [36]:
print(len(cut_no_val))

49


In [34]:
cut_no_test

array([138,  12,  24, 102, 113, 104, 114, 158,  21,   7,  58,   4, 109,
       134,  63,  28,  23, 120,  93,  37,   3,  75, 163,  70,  99,  71,
       117, 128,  76,  52,  39, 129,  25,   2, 108,  69, 137, 124,  49,
        22, 135, 146,  67, 121,  74, 148, 119, 139, 103,  54])

# milling_select_features

In [3]:
from src.features.select_feat_and_scale import milling_select_features, milling_add_y_label_anomaly, save_feat_dict

In [4]:
df = pd.read_csv(folder_processed_data_milling / "milling_features.csv.gz", compression='gzip',)
print(df.shape)

(11570, 4510)


In [5]:
df.head()

,cut_id,ae_spindle__time_reversal_asymmetry_statistic__lag_1,ae_spindle__time_reversal_asymmetry_statistic__lag_2,ae_spindle__time_reversal_asymmetry_statistic__lag_3,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__cid_ce__normalize_False,ae_spindle__symmetry_looking__r_0.0,...,smcac__query_similarity_count__query_None__threshold_0.0,"smcac__matrix_profile__feature_""min""__threshold_0.98","smcac__matrix_profile__feature_""max""__threshold_0.98","smcac__matrix_profile__feature_""mean""__threshold_0.98","smcac__matrix_profile__feature_""median""__threshold_0.98","smcac__matrix_profile__feature_""25""__threshold_0.98","smcac__matrix_profile__feature_""75""__threshold_0.98",cut_no,case,tool_class
0,50_23,-0.000700,-0.000988,-0.000769,0.041818,0.040791,0.040748,7.401132,0.350542,0.0,...,NaN,1.233436,10.531266,3.480118,1.316036,1.283603,5.583248,50,4,1
1,50_24,-0.000536,-0.001016,-0.001079,0.034839,0.033864,0.034001,7.615166,0.387774,0.0,...,NaN,2.983243,13.564660,9.777620,10.404543,6.965601,13.564660,50,4,1
2,50_25,0.000332,-0.000274,-0.000047,0.046048,0.044843,0.044398,8.232772,0.415797,0.0,...,NaN,0.953220,1.225255,1.059743,1.047927,1.008660,1.102001,50,4,1
3,50_26,-0.000724,-0.001458,-0.001034,0.038401,0.037062,0.036612,6.549630,0.342792,0.0,...,NaN,1.323206,11.752717,4.691826,3.096905,1.420717,8.487085,50,4,1
4,50_27,0.000632,0.001454,0.002155,0.042605,0.041493,0.041111,7.367592,0.381698,0.0,...,NaN,1.332938,13.266499,7.684165,8.606933,3.320817,11.341327,50,4,1


In [6]:
df_train, df_val, df_test = milling_select_features(df, random_seed=76, train_size_pct=0.40)

Original shape of df_feat: (11570, 4510)
Final df_train shape: (4674, 1144) (40.40% of samples)
Final df_val shape: (3429, 1144) (29.64% of samples)
Final df_test shape: (3467, 1144) (29.97% of samples)


In [15]:
df_train.reset_index()[['cut_id', 'case', 'tool_class', 'y' ]].to_numpy()

,cut_id,case,tool_class,y
0,50_23,4,1,0
1,50_24,4,1,0
2,50_25,4,1,0
3,50_26,4,1,0
4,50_27,4,1,0
...,...,...,...,...
4669,162_64,16,0,0
4670,162_65,16,0,0
4671,162_7,16,0,0
4672,162_8,16,0,0


In [7]:
dfy = df_train.reset_index()
dfy.head()

,cut_id,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2",...,"ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0",case,tool_class,y
0,50_23,0.228632,7.401132,0.572352,0.572352,0.270641,4.147273,0.171175,0.022750,0.889350,...,3.475581,0.001809,1.0,0.078260,-0.004730,2.142089,0.000552,4,1,0
1,50_24,0.228632,7.615166,0.543775,0.543775,0.404510,1.831587,0.082943,0.033327,0.767750,...,3.984393,-0.026213,1.0,0.146273,-0.000567,2.001063,0.000392,4,1,0
2,50_25,0.228632,8.232772,0.430527,0.430527,0.404510,6.258402,0.015655,0.026201,0.746783,...,4.187977,0.045843,0.0,0.093430,0.007935,2.043632,0.000945,4,1,0
3,50_26,0.228632,6.549630,0.662450,0.662450,0.270641,5.168649,0.277326,0.029816,0.804893,...,3.595821,0.279565,1.0,0.134507,0.005123,2.219708,0.000585,4,1,0
4,50_27,0.228632,7.367592,0.561258,0.561258,0.270641,2.824470,0.072055,0.041497,0.975342,...,3.641927,-0.043193,1.0,0.085082,0.009918,1.988028,0.000591,4,1,0


In [9]:
y = dfy[['cut_id', 'case', 'tool_class', 'y']].to_numpy()

In [10]:
y

array([['50_23', 4, 1, 0],
       ['50_24', 4, 1, 0],
       ['50_25', 4, 1, 0],
       ...,
       ['162_7', 16, 0, 0],
       ['162_8', 16, 0, 0],
       ['162_9', 16, 0, 0]], dtype=object)

In [11]:
y.shape

(4674, 4)

In [14]:
y[:,3].astype(int)

array([0, 0, 0, ..., 0, 0, 0])

In [27]:
df_train.reset_index(inplace=True)
df_train.head()

,cut_id,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2",...,"ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0",case,tool_class,y
0,50_23,0.228632,7.401132,0.572352,0.572352,0.270641,4.147273,0.171175,0.022750,0.889350,...,3.475581,0.001809,1.0,0.078260,-0.004730,2.142089,0.000552,4,1,0
1,50_24,0.228632,7.615166,0.543775,0.543775,0.404510,1.831587,0.082943,0.033327,0.767750,...,3.984393,-0.026213,1.0,0.146273,-0.000567,2.001063,0.000392,4,1,0
2,50_25,0.228632,8.232772,0.430527,0.430527,0.404510,6.258402,0.015655,0.026201,0.746783,...,4.187977,0.045843,0.0,0.093430,0.007935,2.043632,0.000945,4,1,0
3,50_26,0.228632,6.549630,0.662450,0.662450,0.270641,5.168649,0.277326,0.029816,0.804893,...,3.595821,0.279565,1.0,0.134507,0.005123,2.219708,0.000585,4,1,0
4,50_27,0.228632,7.367592,0.561258,0.561258,0.270641,2.824470,0.072055,0.041497,0.975342,...,3.641927,-0.043193,1.0,0.085082,0.009918,1.988028,0.000591,4,1,0


In [28]:
df_train['y'].to_numpy()

array([0, 0, 0, ..., 0, 0, 0])

In [19]:
df_train

,index,cut_id,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40",...,"ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0",case,tool_class,y
0,0,50_23,0.228632,7.401132,0.572352,0.572352,0.270641,4.147273,0.171175,0.022750,...,3.475581,0.001809,1.0,0.078260,-0.004730,2.142089,0.000552,4,1,0
1,1,50_24,0.228632,7.615166,0.543775,0.543775,0.404510,1.831587,0.082943,0.033327,...,3.984393,-0.026213,1.0,0.146273,-0.000567,2.001063,0.000392,4,1,0
2,2,50_25,0.228632,8.232772,0.430527,0.430527,0.404510,6.258402,0.015655,0.026201,...,4.187977,0.045843,0.0,0.093430,0.007935,2.043632,0.000945,4,1,0
3,3,50_26,0.228632,6.549630,0.662450,0.662450,0.270641,5.168649,0.277326,0.029816,...,3.595821,0.279565,1.0,0.134507,0.005123,2.219708,0.000585,4,1,0
4,4,50_27,0.228632,7.367592,0.561258,0.561258,0.270641,2.824470,0.072055,0.041497,...,3.641927,-0.043193,1.0,0.085082,0.009918,1.988028,0.000591,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4669,4669,162_64,0.228632,11.403611,-0.033324,-0.033324,0.228632,0.000000,-0.197102,0.101506,...,4.138482,-0.044998,1.0,0.149315,0.002332,2.062568,0.000107,16,0,0
4670,4670,162_65,0.228632,12.409701,-0.211983,-0.211983,0.270641,0.000000,-0.226678,0.101104,...,4.211430,-0.150215,1.0,0.089608,0.001766,2.192110,0.000213,16,0,0
4671,4671,162_7,0.228632,10.974537,0.032567,0.032567,0.270641,0.000000,-0.074033,0.107358,...,4.335329,0.154681,1.0,0.141489,0.003749,2.049349,0.000139,16,0,0
4672,4672,162_8,0.228632,11.525933,-0.044469,-0.044469,0.270641,0.000000,-0.234196,0.094919,...,3.961558,0.098981,1.0,0.125392,0.000283,1.954515,0.000099,16,0,0


In [25]:
df_train = df_train.reset_index(inplace=True)
y_train = df_train.to_numpy()

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [15]:
df_train

,cut_id,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2",...,"ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0",case,tool_class,y
0,50_23,0.228632,7.401132,0.572352,0.572352,0.270641,4.147273,0.171175,0.022750,0.889350,...,3.475581,0.001809,1.0,0.078260,-0.004730,2.142089,0.000552,4,1,0
1,50_24,0.228632,7.615166,0.543775,0.543775,0.404510,1.831587,0.082943,0.033327,0.767750,...,3.984393,-0.026213,1.0,0.146273,-0.000567,2.001063,0.000392,4,1,0
2,50_25,0.228632,8.232772,0.430527,0.430527,0.404510,6.258402,0.015655,0.026201,0.746783,...,4.187977,0.045843,0.0,0.093430,0.007935,2.043632,0.000945,4,1,0
3,50_26,0.228632,6.549630,0.662450,0.662450,0.270641,5.168649,0.277326,0.029816,0.804893,...,3.595821,0.279565,1.0,0.134507,0.005123,2.219708,0.000585,4,1,0
4,50_27,0.228632,7.367592,0.561258,0.561258,0.270641,2.824470,0.072055,0.041497,0.975342,...,3.641927,-0.043193,1.0,0.085082,0.009918,1.988028,0.000591,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4669,162_64,0.228632,11.403611,-0.033324,-0.033324,0.228632,0.000000,-0.197102,0.101506,1.140711,...,4.138482,-0.044998,1.0,0.149315,0.002332,2.062568,0.000107,16,0,0
4670,162_65,0.228632,12.409701,-0.211983,-0.211983,0.270641,0.000000,-0.226678,0.101104,1.233588,...,4.211430,-0.150215,1.0,0.089608,0.001766,2.192110,0.000213,16,0,0
4671,162_7,0.228632,10.974537,0.032567,0.032567,0.270641,0.000000,-0.074033,0.107358,1.161041,...,4.335329,0.154681,1.0,0.141489,0.003749,2.049349,0.000139,16,0,0
4672,162_8,0.228632,11.525933,-0.044469,-0.044469,0.270641,0.000000,-0.234196,0.094919,1.179877,...,3.961558,0.098981,1.0,0.125392,0.000283,1.954515,0.000099,16,0,0


In [7]:
from sklearn import preprocessing   

In [9]:
scaler = preprocessing.StandardScaler().fit(df_train.drop(['y', 'cut_no', 'tool_class'], axis=1))

In [10]:
X_scaled = scaler.transform(df_train.drop(['y', 'cut_no', 'tool_class'], axis=1))

In [12]:
X_scaled.shape

(4674, 1141)

In [27]:
df_train.reset_index(inplace=True)
df_train.drop(['cut_id'], axis=1, inplace=True)
df_train.head()

,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",...,"ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0",case,tool_class,y
0,0.228632,7.401132,0.572352,0.572352,0.270641,4.147273,0.171175,0.022750,0.889350,0.740909,...,3.475581,0.001809,1.0,0.078260,-0.004730,2.142089,0.000552,4,1,0
1,0.228632,7.615166,0.543775,0.543775,0.404510,1.831587,0.082943,0.033327,0.767750,0.722966,...,3.984393,-0.026213,1.0,0.146273,-0.000567,2.001063,0.000392,4,1,0
2,0.228632,8.232772,0.430527,0.430527,0.404510,6.258402,0.015655,0.026201,0.746783,0.723238,...,4.187977,0.045843,0.0,0.093430,0.007935,2.043632,0.000945,4,1,0
3,0.228632,6.549630,0.662450,0.662450,0.270641,5.168649,0.277326,0.029816,0.804893,0.685764,...,3.595821,0.279565,1.0,0.134507,0.005123,2.219708,0.000585,4,1,0
4,0.228632,7.367592,0.561258,0.561258,0.270641,2.824470,0.072055,0.041497,0.975342,0.755208,...,3.641927,-0.043193,1.0,0.085082,0.009918,1.988028,0.000591,4,1,0


In [29]:
"cut_id" in df_train.columns

False

In [9]:
from tsfresh import select_features, feature_extraction

In [14]:
"cut_no" in col_selected

True

In [19]:
feature_extraction.settings.from_columns(df_train.columns, columns_to_ignore=['cut_no', 'tool_class', 'y', 'case'])

{'smcac': {'fourier_entropy': [{'bins': 2},
   {'bins': 10},
   {'bins': 5},
   {'bins': 100},
   {'bins': 3}],
  'agg_autocorrelation': [{'f_agg': 'median', 'maxlag': 40},
   {'f_agg': 'mean', 'maxlag': 40},
   {'f_agg': 'var', 'maxlag': 40}],
  'change_quantiles': [{'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.8, 'ql': 0.2},
   {'f_agg': 'var', 'isabs': False, 'qh': 0.8, 'ql': 0.0},
   {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4},
   {'f_agg': 'mean', 'isabs': True, 'qh': 0.6, 'ql': 0.0},
   {'f_agg': 'var', 'isabs'

In [23]:
save_feat_dict(df_train, ['cut_no', 'tool_class', 'y', 'case'], folder_processed_data_milling)

In [16]:
len(col_selected)

1141

In [20]:
# assert that df_train and df_val have the same column names
assert set(df_train.columns) == set(df_test.columns)

In [21]:
df_test.head()

,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",...,"ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0",case,tool_class,y
cut_id,,,,,,,,,,,,,,,,,,,,,
52_0,0.135794,8.470622,0.425779,0.425779,0.40451,4.819686,0.051989,-0.000081,0.612793,0.538621,...,2.914282,0.043608,1.0,0.115522,0.003292,2.168505,0.000106,9,0,0
52_1,0.135794,9.308869,0.313841,0.313841,0.40451,6.596196,-0.048826,-0.034267,0.643175,0.507735,...,2.861240,0.022114,1.0,0.089918,0.003095,2.083714,0.000134,9,0,0
52_10,0.135794,9.495586,0.275725,0.275725,0.40451,5.384562,0.175878,-0.002572,0.611213,0.485297,...,3.040242,-0.003837,1.0,0.078974,0.001243,2.064354,0.000104,9,0,0
52_11,0.135794,8.030219,0.498892,0.498892,0.40451,5.384181,0.072003,-0.013781,0.598530,0.516299,...,2.885893,0.267218,1.0,0.066127,0.002158,2.033296,0.000096,9,0,0
52_12,0.135794,8.061453,0.473816,0.473816,0.40451,6.646848,0.093239,-0.010661,0.660722,0.491304,...,2.948825,0.145464,0.0,0.157285,0.002529,2.205641,0.000125,9,0,0


In [101]:
a = list(df_val.columns)
print(a)

['smcac__fourier_entropy__bins_2', 'ae_spindle__cid_ce__normalize_True', 'ae_spindle__partial_autocorrelation__lag_1', 'ae_spindle__autocorrelation__lag_1', 'smcac__fourier_entropy__bins_10', 'smcdc__fft_coefficient__attr_"abs"__coeff_1', 'ae_spindle__autocorrelation__lag_2', 'smcac__agg_autocorrelation__f_agg_"median"__maxlag_40', 'smcac__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2', 'smcac__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0', 'smcac__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.2', 'smcac__cid_ce__normalize_False', 'smcac__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0', 'smcac__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.2', 'smcac__change_quantiles__f_agg_"var"__isabs_False__qh_0.6__ql_0.0', 'smcac__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.2', 'smcac__change_quantiles__f_agg_"mean"__isabs_True__qh_0.8__ql_0.0', 'smcac__autocorrelation__lag_5', 'smcac__fft_coefficient__attr_"abs"__coeff_4', 

In [96]:
df_val

,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",...,vib_table__binned_entropy__max_bins_10,"smcdc__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.2","ae_table__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0","ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0"
cut_id,,,,,,,,,,,,,,,,,,,,,
53_0,0.135794,8.138119,0.476155,0.476155,0.404510,4.792557,0.105344,0.001330,0.718452,0.604074,...,2.130290,-0.001221,0.000309,3.542919,0.166205,1.0,0.120762,0.002812,2.054707,0.000215
53_1,0.135794,7.224293,0.599476,0.599476,0.404510,4.354870,0.092847,-0.019119,0.620117,0.522810,...,2.179124,-0.009155,0.000271,2.931822,0.234366,0.0,0.080618,0.005493,2.168854,0.000155
53_10,0.135794,9.115501,0.344688,0.344688,0.404510,5.930383,0.041085,-0.021053,0.598288,0.496109,...,2.218967,-0.010742,0.000158,2.977242,0.184827,0.0,0.101323,0.004120,2.066499,0.000130
53_11,0.135794,7.900020,0.515823,0.515823,0.404510,5.609229,0.158972,0.002436,0.643095,0.549588,...,2.183841,0.003798,0.000399,3.276091,0.005360,1.0,0.096113,0.010507,1.959778,0.000190
53_12,0.135794,9.519356,0.286779,0.286779,0.404510,4.501335,-0.009667,-0.024678,0.637850,0.519826,...,2.202624,-0.001395,0.000115,3.100451,0.128838,1.0,0.104126,0.005973,2.174034,0.000173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166_64,0.228632,9.871339,0.160403,0.160403,0.228632,0.000000,-0.020191,0.163641,1.192555,1.072746,...,1.995187,0.000000,0.000000,6.544331,-0.074399,1.0,0.135630,-0.001221,1.975564,0.000293
166_65,0.228632,9.866497,0.216034,0.216034,0.228632,0.000000,-0.036824,0.162123,1.438802,1.105453,...,2.121002,0.000000,0.000006,5.766136,-0.027036,0.0,0.120621,-0.000305,1.969582,0.000379
166_7,0.228632,11.050184,0.035192,0.035192,0.270641,0.000000,-0.106145,0.157030,1.404239,1.072514,...,2.062320,0.000000,0.000000,6.996343,-0.141783,0.0,0.097623,-0.000698,2.049519,0.000438


In [82]:
df_feat_sel.head()

,smcac__fourier_entropy__bins_2,ae_spindle__cid_ce__normalize_True,ae_spindle__partial_autocorrelation__lag_1,ae_spindle__autocorrelation__lag_1,smcac__fourier_entropy__bins_10,"smcdc__fft_coefficient__attr_""abs""__coeff_1",ae_spindle__autocorrelation__lag_2,"smcac__agg_autocorrelation__f_agg_""median""__maxlag_40","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_0.6__ql_0.2","smcac__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.0",...,vib_table__binned_entropy__max_bins_10,"smcdc__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.2","ae_table__change_quantiles__f_agg_""var""__isabs_False__qh_0.2__ql_0.0","ae_spindle__fft_aggregated__aggtype_""centroid""",ae_table__ar_coefficient__coeff_6__k_10,vib_spindle__large_standard_deviation__r_0.2,smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0,"ae_spindle__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""min""",ae_spindle__binned_entropy__max_bins_10,"ae_spindle__change_quantiles__f_agg_""var""__isabs_True__qh_0.8__ql_0.0"
cut_id,,,,,,,,,,,,,,,,,,,,,
50_23,0.228632,7.401132,0.572352,0.572352,0.270641,4.147273,0.171175,0.022750,0.889350,0.740909,...,2.219080,0.000814,0.000638,3.475581,0.001809,1.0,0.078260,-0.004730,2.142089,0.000552
50_24,0.228632,7.615166,0.543775,0.543775,0.404510,1.831587,0.082943,0.033327,0.767750,0.722966,...,2.238344,-0.009766,0.000030,3.984393,-0.026213,1.0,0.146273,-0.000567,2.001063,0.000392
50_25,0.228632,8.232772,0.430527,0.430527,0.404510,6.258402,0.015655,0.026201,0.746783,0.723238,...,2.218349,-0.021701,0.000469,4.187977,0.045843,0.0,0.093430,0.007935,2.043632,0.000945
50_26,0.228632,6.549630,0.662450,0.662450,0.270641,5.168649,0.277326,0.029816,0.804893,0.685764,...,2.257441,-0.006975,0.000139,3.595821,0.279565,1.0,0.134507,0.005123,2.219708,0.000585
50_27,0.228632,7.367592,0.561258,0.561258,0.270641,2.824470,0.072055,0.041497,0.975342,0.755208,...,2.238223,0.010579,0.000120,3.641927,-0.043193,1.0,0.085082,0.009918,1.988028,0.000591


In [88]:
df.drop(columns=a)

KeyError: '[\'ae_spindle__number_crossing_m__m_1\' \'vib_spindle__count_below__t_0\'\n \'ae_spindle__fft_coefficient__attr_"abs"__coeff_88\' ...\n \'ae_spindle__fft_coefficient__attr_"angle"__coeff_35\'\n \'ae_spindle__cid_ce__normalize_False\'\n \'smcdc__fft_coefficient__attr_"abs"__coeff_97\'] not found in axis'

In [91]:
a = list(set(list(df.columns)) - set(list(df_feat_sel.columns)))
a

['ae_spindle__number_crossing_m__m_1',
 'vib_spindle__count_below__t_0',
 'ae_spindle__fft_coefficient__attr_"abs"__coeff_88',
 'ae_spindle__fft_coefficient__attr_"angle"__coeff_5',
 'vib_spindle__fft_coefficient__attr_"imag"__coeff_33',
 'ae_table__fft_coefficient__attr_"abs"__coeff_62',
 'ae_spindle__fft_coefficient__attr_"real"__coeff_36',
 'smcac__fft_coefficient__attr_"real"__coeff_22',
 'ae_table__fft_coefficient__attr_"real"__coeff_77',
 'smcac__fft_coefficient__attr_"imag"__coeff_36',
 'ae_spindle__fft_coefficient__attr_"abs"__coeff_37',
 'ae_table__fft_coefficient__attr_"real"__coeff_27',
 'ae_table__ratio_beyond_r_sigma__r_6',
 'vib_spindle__fft_coefficient__attr_"abs"__coeff_57',
 'ae_table__fft_coefficient__attr_"imag"__coeff_8',
 'ae_spindle__fft_coefficient__attr_"imag"__coeff_24',
 'ae_table__fft_coefficient__attr_"angle"__coeff_3',
 'ae_spindle__fft_coefficient__attr_"imag"__coeff_37',
 'ae_table__symmetry_looking__r_0.7000000000000001',
 'ae_table__fft_coefficient__att

In [92]:
df.drop(columns=a)

,ae_spindle__c3__lag_1,ae_spindle__c3__lag_2,ae_spindle__c3__lag_3,ae_spindle__cid_ce__normalize_True,ae_spindle__quantile__q_0.4,ae_spindle__quantile__q_0.6,ae_spindle__quantile__q_0.7,ae_spindle__quantile__q_0.8,ae_spindle__quantile__q_0.9,ae_spindle__autocorrelation__lag_1,...,smcac__count_above__t_0,smcac__count_below__t_0,smcac__lempel_ziv_complexity__bins_3,smcac__fourier_entropy__bins_2,smcac__fourier_entropy__bins_3,smcac__fourier_entropy__bins_5,smcac__fourier_entropy__bins_10,smcac__fourier_entropy__bins_100,smcac__permutation_entropy__dimension_3__tau_1,smcac__permutation_entropy__dimension_4__tau_1
0,0.041818,0.040791,0.040748,7.401132,0.338135,0.353760,0.369446,0.392334,0.401245,0.572352,...,0.515625,0.484375,0.265625,0.228632,0.270641,0.270641,0.270641,0.537369,1.343274,1.956743
1,0.034839,0.033864,0.034001,7.615166,0.310547,0.335815,0.348084,0.360474,0.383240,0.543775,...,0.406250,0.593750,0.281250,0.228632,0.270641,0.270641,0.404510,0.627174,1.267146,1.763141
2,0.046048,0.044843,0.044398,8.232772,0.334473,0.369507,0.391357,0.400024,0.415039,0.430527,...,0.453125,0.546875,0.265625,0.228632,0.270641,0.270641,0.404510,0.404510,1.380839,1.924768
3,0.038401,0.037062,0.036612,6.549630,0.310059,0.353394,0.373047,0.382568,0.399292,0.662450,...,0.562500,0.437500,0.265625,0.228632,0.270641,0.270641,0.270641,0.537369,1.260400,1.770192
4,0.042605,0.041493,0.041111,7.367592,0.329834,0.353027,0.373413,0.394531,0.413635,0.561258,...,0.484375,0.515625,0.265625,0.228632,0.270641,0.270641,0.270641,0.404510,1.307331,1.781249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11565,0.014140,0.013805,0.014852,9.871339,0.219238,0.250732,0.256409,0.275024,0.313782,0.160403,...,0.406250,0.593750,0.281250,0.228632,0.228632,0.228632,0.228632,0.757905,1.277265,1.767133
11566,0.014126,0.013593,0.014314,9.866497,0.225830,0.243164,0.260742,0.272095,0.310730,0.216034,...,0.515625,0.484375,0.296875,0.228632,0.228632,0.228632,0.228632,0.799914,1.196875,1.691628
11567,0.012772,0.012246,0.013285,11.050184,0.209473,0.235229,0.246948,0.276978,0.318542,0.035192,...,0.562500,0.437500,0.265625,0.228632,0.228632,0.228632,0.270641,0.627174,1.204097,1.698747
11568,0.010777,0.010660,0.011797,10.989757,0.201050,0.226685,0.245667,0.262207,0.279175,0.026847,...,0.437500,0.562500,0.265625,0.228632,0.228632,0.228632,0.270641,0.669183,1.517038,2.246702


In [83]:
df_feat_sel.columns

Index(['smcac__fourier_entropy__bins_2', 'ae_spindle__cid_ce__normalize_True',
       'ae_spindle__partial_autocorrelation__lag_1',
       'ae_spindle__autocorrelation__lag_1', 'smcac__fourier_entropy__bins_10',
       'smcdc__fft_coefficient__attr_"abs"__coeff_1',
       'ae_spindle__autocorrelation__lag_2',
       'smcac__agg_autocorrelation__f_agg_"median"__maxlag_40',
       'smcac__change_quantiles__f_agg_"mean"__isabs_True__qh_0.6__ql_0.2',
       'smcac__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.0',
       ...
       'vib_table__binned_entropy__max_bins_10',
       'smcdc__change_quantiles__f_agg_"mean"__isabs_False__qh_0.4__ql_0.2',
       'ae_table__change_quantiles__f_agg_"var"__isabs_False__qh_0.2__ql_0.0',
       'ae_spindle__fft_aggregated__aggtype_"centroid"',
       'ae_table__ar_coefficient__coeff_6__k_10',
       'vib_spindle__large_standard_deviation__r_0.2',
       'smcac__energy_ratio_by_chunks__num_segments_10__segment_focus_0',
       'ae_spindle__a